<a href="https://colab.research.google.com/github/sonenhur/k5_pytorch/blob/main/PyTorch_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Working with data

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [ ]:
batch_size = 128

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([128, 1, 28, 28])
Shape of y: torch.Size([128]) torch.int64


In [ ]:
for i in [1,2,3]:
  print(i)
for s in "hello":
  print(s)
for i in range(5):
  print(i)

1
2
3
h
e
l
l
o
0
1
2
3
4


## Creating Models

### Get cpu, gpu or mps device for training

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
a = max
type(a)

builtin_function_or_method

In [ ]:
a = range(5)
type(a)

range

### Define model

In [ ]:
class NeuralNetwork(nn.Module): #nn,Module()을 상속
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(type(model)) # 타입 확인 습관화
# 여기서는 함수호출이 아니라 모듈 객체를 생성함

<class '__main__.NeuralNetwork'>


In [ ]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

In [ ]:
loss_fn = nn.CrossEntropyLoss() # 손실함수: Classification(분류)에 사용}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) #(10의 -4승)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300760  [  128/60000]
loss: 0.761876  [12928/60000]
loss: 0.562215  [25728/60000]
loss: 0.553815  [38528/60000]
loss: 0.549479  [51328/60000]
Test Error: 
 Accuracy: 81.1%, Avg loss: 0.527106 

Epoch 2
-------------------------------
loss: 0.382980  [  128/60000]
loss: 0.396149  [12928/60000]
loss: 0.381702  [25728/60000]
loss: 0.452636  [38528/60000]
loss: 0.471568  [51328/60000]
Test Error: 
 Accuracy: 83.5%, Avg loss: 0.463826 

Epoch 3
-------------------------------
loss: 0.285686  [  128/60000]
loss: 0.341436  [12928/60000]
loss: 0.334203  [25728/60000]
loss: 0.418491  [38528/60000]
loss: 0.430041  [51328/60000]
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.435624 

Epoch 4
-------------------------------
loss: 0.249471  [  128/60000]
loss: 0.311197  [12928/60000]
loss: 0.309382  [25728/60000]
loss: 0.396011  [38528/60000]
loss: 0.408548  [51328/60000]
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.417309 

Epoch 5
------------------------

## Saving Models

In [ ]:
scripted_model = torch.jit.script(model)
scripted_model.save('fmnist_model.pt')

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

### This model can now be used to make predictions.

In [ ]:
classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot",]

In [ ]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
